In [2]:
import numpy as np
import matplotlib.pyplot as plt

from typing import Callable, Any

### FireFly Partical

In [233]:
class FAParticle:

    def __init__(self, id: int = 0, dim: tuple[int] = 2):
        self.dim = dim
        self.id = id
        self._pos = np.random.rand(dim)

    @property
    def pos(self):
        return self._pos

    @pos.setter
    def pos(self, pos: np.ndarray):
        self._pos = pos        

### FireFly Algorithm

In [266]:
class FireFly:

    def __init__(
        self,
        beta0: float = 1,
        gamma: float = 0.01, 
        delta: float = 0.97, 
        pop_size: int = 10,
        max_iters: int = 100,
        dim: int = 2,
        eps: float = 1e-3
    ) -> None:

        self.beta0 = beta0
        self.gamma = gamma
        self.delta = delta
        self.pop_size = pop_size
        self.max_iters = max_iters
        self.dim: int = dim
        self.eps = eps
        self.func: Callable[[Any], Any] = None

        self.alpha0 = np.random.rand(self.dim)
        self.alpha = self.alpha0 * self.delta  # Initialize alpha once
        
        self.fireflies: List[FAParticle] = [FAParticle(id=i, dim=dim) for i in range(self.pop_size)]
        self.intensity: np.ndarray = None

    def run(self, func: Callable[[Any], Any]):
        self.func = func

        self.intensity = np.array([self.func(fa.pos) for fa in self.fireflies])
        
        arg_best = np.argmin(self.intensity)
        best = self.intensity[arg_best]
        
        t = 0
        while t <= self.max_iters:

            self.alpha *= self.delta
            
            for i, fi in enumerate(self.fireflies):

                for j, fj in enumerate(self.fireflies):

                    if self.intensity[i] > self.intensity[j]:

                        r = np.sum(np.square(self.fireflies[i].pos - self.fireflies[j].pos))

                        beta = self.beta0 * np.exp(-self.gamma * r)

                        self.fireflies[i].pos += beta * (self.fireflies[j].pos - self.fireflies[i].pos) + self.alpha
                        
                        self.intensity[i] = self.func(self.fireflies[i].pos)
                        
            arg_best = np.argmin(self.intensity)
            best = self.intensity[arg_best]
            
            t += 1
        
        return arg_best, best, self.fireflies[arg_best].pos

In [267]:
def func(vec: np.ndarray):
    x, y = vec
    return (x + 2*y - 7)**2 + (2*x + y - 5)**2

In [268]:
func(np.array([1, 3]))

0

In [271]:
firefly = FireFly(max_iters=1000, pop_size=50)

In [272]:
firefly.run(func)

(31, 4.681467899337669, array([2.54409525, 1.48549141]))